In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#loading data
data = pd.read_csv('../input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv')
data

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
#cleaning the data
data.isnull().sum()

## relationship analysis

In [ ]:
cor = data.corr()
cor

In [ ]:
sns.heatmap(cor, xticklabels = cor.columns, yticklabels = cor.columns, annot = True)

In [ ]:
sns.pairplot(data)

In [ ]:
sns.relplot(x = 'Total Cases', y ='Deaths', hue = 'State/UTs', data = data)

In [ ]:
sns.relplot(x = 'Total Cases', y ='Active',hue = 'State/UTs', data = data)

In [ ]:
sns.relplot(x = 'Total Cases', y ='Discharged',hue = 'State/UTs', data = data)

In [ ]:
from geopy.geocoders import Nominatim


In [ ]:
locations = pd.DataFrame()

In [ ]:
locations =  data
locations.head()

In [ ]:
locations =  locations.rename(columns = {"State/UTs":"State"})
locations.head()

In [ ]:
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['State']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

In [ ]:
locations['geo_loc']=lat_lon


In [ ]:
locations.head()

### We have found out latitude and longitude of each location listed in the dataset using geopy
### This is used to plot maps.

In [ ]:
State_locations=pd.DataFrame(data['State/UTs'].value_counts().reset_index())

In [ ]:
State_locations.columns=['State','count']
State_locations.head()

### now combine both the dataframes

In [ ]:
locations.head()

In [ ]:
locations.shape

In [ ]:
STS_locations=State_locations.merge(locations,on='State',how="left").dropna()
STS_locations.head()

In [ ]:
STS_locations.shape

In [ ]:
type(STS_locations['geo_loc'][0])

In [ ]:
def generateBaseMap(default_location=[20.59,78.96], default_zoom_start=5):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [ ]:
import folium
from folium.plugins import HeatMap
basemap=generateBaseMap()

In [ ]:
basemap

In [ ]:
STS_locations['geo_loc'][0][0]

In [ ]:
STS_locations['geo_loc'][0][1]

In [ ]:
np.array(STS_locations['geo_loc'])

In [ ]:
lat,lon=zip(*np.array(STS_locations['geo_loc']))

In [ ]:
STS_locations['lat']=lat
STS_locations['lon']=lon

In [ ]:
STS_locations.head()

In [ ]:
STS_locations[['lat','lon','Active']].values.tolist()

## Geographical Analysis For Active Cases:

In [ ]:
HeatMap(STS_locations[['lat','lon','Active']].values.tolist(),zoom=20,radius=15,cmap="Greens").add_to(basemap)

In [ ]:
basemap

## Geographical Analysis For Death Cases:

In [ ]:
HeatMap(STS_locations[['lat','lon','Deaths']].values.tolist(),zoom=20,radius=15,cmap="Greens").add_to(basemap)

In [ ]:
basemap

## Geographical Analysis For Discharge Cases:

In [ ]:
HeatMap(STS_locations[['lat','lon','Discharged']].values.tolist(),zoom=20,radius=15,cmap="Greens").add_to(basemap)

In [ ]:
basemap